In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

text = "과자"
driver = webdriver.Chrome()
driver.get(f'https://msearch.shopping.naver.com/search/all?query={text}&frm=NVSHSRC&vertical=search')
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
time.sleep(5)
a = len(driver.find_elements(By.CSS_SELECTOR, ".product_list_item__b84TO")) 
print(a)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
posts = soup.select(".product_list_item__b84TO")
# print(posts)
data = []
for post in posts:
    # time.sleep(3)
    title = post.select_one(".product_info_tit__c5_pb").text
    link = post.select_one(".product_btn_link__ArGCa.linkAnchor").attrs['href']
    money = post.select_one(".product_num__iQsWh").text
    data.append([title, money, link])
df =  pd.DataFrame(data, columns = ['제목','가격','링크'])
df.to_excel('./aaa.xlsx')

12


In [1]:
from openpyxl import Workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

# 데이터 로드
df = pd.read_excel('./aaa1.xlsx')

wb = Workbook()
ws = wb.active

# dataframe_to_rows 함수는 기본적으로 첫 번째 행에 헤더를 넣습니다.
# 따라서 r_idx > 1 조건을 사용하여 첫 번째 행(헤더)을 건너뛸 수 있습니다.
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
    if r_idx > 1:  # 첫 번째 행(헤더)를 건너뜀
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            if c_idx == 2:  # 'link' 컬럼을 기준으로 하이퍼링크 설정
                cell.font = Font(color="0000FF", underline="single")
                cell.hyperlink = row[3]  # 'link'의 값을 이용, 컬럼 인덱스는 0부터 시작하므로 '1'을 사용

wb.save("./output.xlsx")
